In [1]:
from sympy import *
from IPython.display import display
init_printing(use_latex='mathjax')

# Method of Multiple Scales

The method of multiple scales is a technique used to construct uniformly valid approximations to the perturbed solutions, both for small and large values of the independent variables. It is particularly useful in systems characterized by disparate time scales, such as when weak effects become significant on long time scales.

## Nonlinear Oscillators and the Poincare-Linstedt Method

The first examples of the method of multiple scales are built upon the nonlinear oscillators, who in their unperturbed state both form the equation $$\ddot{y_0} + y_0 = 0,$$ whose solutions are some sinusoid $A\cos(\omega t+\phi),$ which has frequency $\omega = 1$. Lower orders are often described by some forced equation $$\ddot{y_1} + y_1 = F(y_0).$$ Since $y_0$ is a sinusoid, there is a reasonable chance that the forcing will resonate, resulting in a solution of the form $t\cos(t),$ which on a sufficiently large time scale will come to dominate the behavior. But this is not what is observed, and for oscillators, this is rectified via an asymptotic expansion in $\omega$ and/or the amplitude $A$. This expansion ultimately modifies the resonant terms in $F,$ allowing for a *solvability condition* to emerge so as to nullify the effects of these resonance terms. 

When applied to periodic functions, as we are in these oscillator cases, this is known as the Poincare-Linstedt Method.

### Duffing's equation

Our first example comes from Duffing's nonlinear oscillator $$\ddot{y} + y + \epsilon y^3,$$ with $y(0)=1, y'(0) = 0.$ The following code solve this equation by a normal asymptotic method from the previous chapter. As we can see, the output contains a *secular term* $t\sin(t)$ that grows linearly in $t$. As such, this approximation is only valid when $t\ll\epsilon^{-1}$.

In [36]:
t, epsilon = symbols('t epsilon')
N = 2
y_ = [Function(f'y_{i}')(t) for i in range(N)]
y = sum([epsilon**i * y_[i] for i in range(N)])
ddt = lambda f: f.diff(t)
eqn = ddt(ddt(y)) + y + epsilon*y**3
eqn = eqn.expand().collect(epsilon,evaluate=False)
display(eqn)
#Define zero ICs/BCs (Sympy puts them all under the term ICs)
ics = []
for i in range(N):
    ics.append({y_[i].subs(t,0):0,y_[i].diff(t).subs(t,0):0})
#update other ICs/BCs
ics[0][y_[0].subs(t,0)] = 1

#solve the ODEs
replacements = []
for i in range(N):
    soln = dsolve(eqn[epsilon**i].subs(replacements),y_[i],ics=ics[i])
    display(soln)
    replacements.append((soln.lhs,soln.rhs))
display(y.subs(replacements))

⎧            2                               2                                 ↪
⎪           d                3              d            2      2            3 ↪
⎨1: y₀(t) + ───(y₀(t)), ε: y₀ (t) + y₁(t) + ───(y₁(t)), ε : 3⋅y₀ (t)⋅y₁(t), ε  ↪
⎪             2                               2                                ↪
⎩           dt                              dt                                 ↪

↪                             ⎫
↪             2      4    3   ⎪
↪ : 3⋅y₀(t)⋅y₁ (t), ε : y₁ (t)⎬
↪                             ⎪
↪                             ⎭

y₀(t) = cos(t)

                                                  5            
        ⎛  3⋅t   sin(2⋅t)   sin(4⋅t)⎞          cos (t)   cos(t)
y₁(t) = ⎜- ─── - ──────── - ────────⎟⋅sin(t) - ─────── + ──────
        ⎝   8       4          32   ⎠             4        4   

  ⎛                                          5            ⎞         
  ⎜⎛  3⋅t   sin(2⋅t)   sin(4⋅t)⎞          cos (t)   cos(t)⎟         
ε⋅⎜⎜- ─── - ──────── - ────────⎟⋅sin(t) - ─────── + ──────⎟ + cos(t)
  ⎝⎝   8       4          32   ⎠             4        4   ⎠         

To construct a valid solution for all time, we introduce a stretched time variable $\tau = \omega(\epsilon)t,$ which implies that $\frac{d}{dt} = \omega\frac{d}{d\tau},$ and by construction $\omega = \omega + \epsilon\omega_1$

In [58]:
epsilon = symbols('epsilon')
tau = symbols('tau')
N = 2
y_ = [Function(f'y_{i}')(tau) for i in range(N)]
y = sum([epsilon**i * y_[i] for i in range(N)])
omega_ = [Symbol(f'omega_{i}')for i in range(N)]
omega = sum([epsilon**i * omega_[i] for i in range(N)])

ddt = lambda f: omega*f.diff(tau)
eqn = ddt(ddt(y)) + y + epsilon*y**3
eqn = eqn.expand().collect(epsilon,evaluate=False)
display(eqn)
#Define zero ICs/BCs (Sympy puts them all under the term ICs)
ics = []
for i in range(N):
    ics.append({y_[i].subs(tau,0):0,y_[i].diff(tau).subs(tau,0):0})
#update other ICs/BCs
ics[0][y_[0].subs(tau,0)] = 1

#solve the ODEs
replacements = []
for i in range(N):
    #display(eqn[epsilon**i].subs(replacements))
    soln = dsolve(eqn[epsilon**i].subs(replacements),y_[i],ics=ics[i])
    display(soln)
    replacements.append((soln.lhs,soln.rhs))
#replacements.extend([(tau,omega*symbols('t')),(omega_[1],Rational(3,8)*omega_[0])])
display(y.subs(replacements).rewrite(cos).simplify())



⎧        2                          2                    2                     ↪
⎪     2 d                        2 d                    d              3       ↪
⎨1: ω₀ ⋅───(y₀(τ)) + y₀(τ), ε: ω₀ ⋅───(y₁(τ)) + 2⋅ω₀⋅ω₁⋅───(y₀(τ)) + y₀ (τ) +  ↪
⎪         2                          2                    2                    ↪
⎩       dτ                         dτ                   dτ                     ↪

↪                     2                2                                    2  ↪
↪         2          d              2 d                2            3    2 d   ↪
↪ y₁(τ), ε : 2⋅ω₀⋅ω₁⋅───(y₁(τ)) + ω₁ ⋅───(y₀(τ)) + 3⋅y₀ (τ)⋅y₁(τ), ε : ω₁ ⋅─── ↪
↪                      2                2                                    2 ↪
↪                    dτ               dτ                                   dτ  ↪

↪                                     ⎫
↪                     2      4    3   ⎪
↪ (y₁(τ)) + 3⋅y₀(τ)⋅y₁ (τ), ε : y₁ (τ)⎬
↪                                     ⎪
↪                           

         ⅈ⋅τ    -ⅈ⋅τ 
         ───    ─────
         ω₀      ω₀  
        ℯ      ℯ     
y₀(τ) = ──── + ──────
         2       2   

                                                                         3⋅ⅈ⋅τ ↪
                                 -ⅈ⋅τ                             ⅈ⋅τ    ───── ↪
                                 ─────                            ───     ω₀   ↪
        ⎛  1    3⋅ⅈ⋅τ   ⅈ⋅ω₁⋅τ⎞   ω₀     ⎛  1    3⋅ⅈ⋅τ   ⅈ⋅ω₁⋅τ⎞  ω₀    ℯ      ↪
y₁(τ) = ⎜- ── - ───── + ──────⎟⋅ℯ      + ⎜- ── + ───── - ──────⎟⋅ℯ    + ────── ↪
        ⎜  64   16⋅ω₀       2 ⎟          ⎜  64   16⋅ω₀       2 ⎟          64   ↪
        ⎝               2⋅ω₀  ⎠          ⎝               2⋅ω₀  ⎠               ↪

↪     -3⋅ⅈ⋅τ 
↪     ───────
↪       ω₀   
↪    ℯ       
↪  + ────────
↪       64   
↪            

  ⎛   ⎛3⋅t⋅(3⋅ε + 8)⎞      ⎛3⋅ε⋅t    ⎞⎞                 
ε⋅⎜cos⎜─────────────⎟ - cos⎜───── + t⎟⎟                 
  ⎝   ⎝      8      ⎠      ⎝  8      ⎠⎠      ⎛3⋅ε⋅t    ⎞
─────────────────────────────────────── + cos⎜───── + t⎟
                  32                         ⎝  8      ⎠

From this, we can observe that the secular term will vanish under the condition that $\omega_1 = \frac38\omega_0,$ and a line of code that makes this substitution. We note that while we utilize this solvability condition as part of the secular term directly, it is also possible to see the solvability condition in the equation, as seen in another commented out line of code.

### Van der Pol Oscillator

While the Duffing Oscillator had a perturbation expansion for the frequency, but the Van der Pol Oscillator also has an expansion in the amplitude. The equation at hand is $$\ddot{y} + \epsilon(y^2-1)y' + y = 0,$$ with $y(0)=a_0 + \epsilon a_1, y'(0) = 0.$ We perform a similar $\omega$ substitution as done in the Duffing Oscillator.

In [60]:
epsilon = symbols('epsilon')
tau = symbols('tau')
N = 2
y_ = [Function(f'y_{i}')(tau) for i in range(N)]
y = sum([epsilon**i * y_[i] for i in range(N)])
omega_ = [Symbol(f'omega_{i}') for i in range(N)]
omega = sum([epsilon**i * omega_[i] for i in range(N)])
a_ = [Symbol(f'a_{i}') for i in range(N)]

ddt = lambda f: omega*f.diff(tau)
eqn = ddt(ddt(y)) + y + epsilon*(y**2-1)*ddt(y)
eqn = eqn.expand().collect(epsilon,evaluate=False)
display(eqn)
#Define zero ICs/BCs (Sympy puts them all under the term ICs)
ics = []
for i in range(N):
    ics.append({y_[i].subs(tau,0):a_[i],y_[i].diff(tau).subs(tau,0):0})
#update other ICs/BCs

#solve the ODEs
replacements = []
for i in range(N):
    soln = dsolve(eqn[epsilon**i].subs(replacements),y_[i],ics=ics[i])
    display(soln)
    replacements.append((soln.lhs,soln.rhs))
replacements.extend([(tau,omega*symbols('t')),(omega_[1],0),(a_[0],2)])
display(y.subs(replacements).rewrite(cos).simplify())

⎧        2                          2                    2                     ↪
⎪     2 d                        2 d                    d                 2    ↪
⎨1: ω₀ ⋅───(y₀(τ)) + y₀(τ), ε: ω₀ ⋅───(y₁(τ)) + 2⋅ω₀⋅ω₁⋅───(y₀(τ)) + ω₀⋅y₀ (τ) ↪
⎪         2                          2                    2                    ↪
⎩       dτ                         dτ                   dτ                     ↪

↪                                                 2                            ↪
↪  d              d                   2          d                 2    d      ↪
↪ ⋅──(y₀(τ)) - ω₀⋅──(y₀(τ)) + y₁(τ), ε : 2⋅ω₀⋅ω₁⋅───(y₁(τ)) + ω₀⋅y₀ (τ)⋅──(y₁( ↪
↪  dτ             dτ                               2                    dτ     ↪
↪                                                dτ                            ↪

↪                                                        2                     ↪
↪                        d              d             2 d                 2    ↪
↪ τ)) + 2⋅ω₀⋅y₀(τ)⋅y₁(τ)⋅─

            ⅈ⋅τ       -ⅈ⋅τ 
            ───       ─────
            ω₀         ω₀  
        a₀⋅ℯ      a₀⋅ℯ     
y₀(τ) = ─────── + ─────────
           2          2    

               3⋅ⅈ⋅τ          -3⋅ⅈ⋅τ                                           ↪
               ─────          ───────                                          ↪
            3   ω₀         3    ω₀      ⎛        3     3                       ↪
        ⅈ⋅a₀ ⋅ℯ        ⅈ⋅a₀ ⋅ℯ          ⎜  7⋅ⅈ⋅a₀    a₀ ⋅τ   ⅈ⋅a₀   a₀⋅τ   ⅈ⋅a ↪
y₁(τ) = ──────────── - ────────────── + ⎜- ─────── - ───── + ──── + ──── - ─── ↪
             64              64         ⎜    64      16⋅ω₀    4     4⋅ω₀       ↪
                                        ⎝                                    2 ↪

↪                                                                            
↪               ⅈ⋅τ                                                     -ⅈ⋅τ 
↪            ⎞  ───   ⎛      3     3                                 ⎞  ─────
↪ ₀⋅ω₁⋅τ   a₁⎟  ω₀    ⎜7⋅ⅈ⋅a₀    a₀ ⋅τ   ⅈ⋅a₀   a₀⋅τ   ⅈ⋅a₀⋅ω₁⋅τ   a₁⎟   ω₀  
↪ ────── + ──⎟⋅ℯ    + ⎜─────── - ───── - ──── + ──── + ───────── + ──⎟⋅ℯ     
↪    2     2 ⎟        ⎜  64      16⋅ω₀    

              3⋅ε⋅sin(t)   ε⋅sin(3⋅t)           
a₁⋅ε⋅cos(t) + ────────── - ────────── + 2⋅cos(t)
                  4            4                

To eliminate the secular terms, we require first that $\omega_1 = 0,$ then we get the condition $a_0^3 - 4a_0 = 0,$ which in turn means that $a_0 = 0,\pm 2.$ We note that the $\pm2$ cases both produce the same solution up to a $180^\circ$ phase offset. $a_1, \omega_2$ can be used to satisfy the next order solvability conditions, as this process can be repeated for the higher orders.

## Method of Multiple Scales (MMS)

While the Poincare-Linstedt method works well for purely periodic problems, it cannot be used when solutions evolve over slow time-scales. MMS works more generally by including several independent variables for each scale of interest. In this section, we will apply the method to time variables, but it can just as easily be applied to spatial variables too, and there may even be cases of multiple time scales and multiple spatial scales; see the notebook entitled IMMD for an example.

Let time $t$ be broken into several time scales, our 'fast' scale $t = t_0$, and slower time scales $t_n = f_n(\epsilon)t_0$ for some natural numbers $n$. One important aspect of this method is that since time now depends on several times scales at once, total derivatives must now be broken into its components. So we get a derivative rule $$\frac{d}{dt} = \partial_{t_0} + \frac{dt_1}{dt_0}\partial_{t_1} + \frac{dt_2}{dt_0}\partial_{t_2} + ...$$ $$\frac{d}{dt} = \partial_{t_0} + f_1(\epsilon)\partial_{t_1} + f_2(\epsilon)\partial_{t_2} + ...,$$
and such expansions apply multiple times for higher order derivaties (e.g. in an wave-like equation)

### Mathieu Equation

We consider the Mathieu equation $$\ddot{y} + (1+\epsilon\delta+\epsilon\cos(kt))y$$ which describes a linear oscillator whose frequency changes sinusoidally. This has been scaled such that $\delta = O(1)$ as $\epsilon\to0$. Here, we will introduce a second time scale $\tau=\epsilon t$, and we prescribe our solutions will take the form $y(t,\tau),$ with the expansion $y = y_0 + \epsilon y_1$. Here we extract the $O(1),O(\epsilon)$ equations.

In [23]:
t, tau, epsilon = symbols('t tau epsilon')
delta, k = symbols('delta k')

N = 2
y_ = [Function(f'y_{i}')(t,epsilon*t) for i in range(N)]
y = sum([epsilon**i * y_[i] for i in range(N)])
ddt = lambda f: f.diff(t)
eqn = ddt(ddt(y)) + (1+epsilon*(delta+cos(k*t)))*y
eqn = eqn.expand().collect(epsilon,evaluate=False)
#define in terms of tau
eqn = {k:v.subs(epsilon*t,tau).simplify() for k,v in eqn.items()}
y_ = [i.subs(epsilon*t,tau) for i in y_]
y = y.subs(epsilon*t,tau)

display(eqn[1],eqn[epsilon])

            2           
           ∂            
y₀(t, τ) + ───(y₀(t, τ))
             2          
           ∂t           

                                             2                  2            
                                            ∂                  ∂             
δ⋅y₀(t, τ) + y₀(t, τ)⋅cos(k⋅t) + y₁(t, τ) + ───(y₁(t, τ)) + 2⋅─────(y₀(t, τ))
                                              2               ∂τ ∂t          
                                            ∂t                               

The solution to this first equation is $y_0 = A(\tau)e^{it} + \overline{A(\tau)}e^{-it},$ for some amplitude function $A$, where the overline denotes the complex conjugate. If we make the substitution into the $O(\epsilon)$ equation under the case $k=2,$ we get the following, with terms separated by their frequency in $t$.

In [63]:
amplitude = Function('A')(tau)
y_0 = amplitude*exp(I*t) + conjugate(amplitude)*exp(-I*t)
display(eqn[epsilon].subs({y_[0]:y_0,k:2})
        .rewrite(exp).simplify().expand()
        .collect(exp(I*t)))

⎛         ____               ⎞                                                 ↪
⎜         A(τ)       d       ⎟  ⅈ⋅t   ⎛  ____   A(τ)       d ⎛____⎞⎞  -ⅈ⋅t   A ↪
⎜δ⋅A(τ) + ──── + 2⋅ⅈ⋅──(A(τ))⎟⋅ℯ    + ⎜δ⋅A(τ) + ──── - 2⋅ⅈ⋅──⎝A(τ)⎠⎟⋅ℯ     + ─ ↪
⎝          2         dτ      ⎠        ⎝          2         dτ      ⎠           ↪
                                                                               ↪

↪      3⋅ⅈ⋅t               2               -3⋅ⅈ⋅t ____
↪ (τ)⋅ℯ                   ∂               ℯ      ⋅A(τ)
↪ ────────── + y₁(t, τ) + ───(y₁(t, τ)) + ────────────
↪     2                     2                  2      
↪                         ∂t                          

Code Note: Sympy currently has trouble running `collect()` with anything that looks like a mixed partial derivative. This is why we do the $\epsilon t\to\tau$ substitutions rather than use $\tau$ directly. Other computer algebra systems likely do not have this specific quirk, but you get what you pay for.

Back to the mathematics, we can identify the coefficient of the resonant terms $$2i\frac{dA}{d\tau} + \delta A(\tau) + \frac12\overline{A(\tau)} + \text{ complex conjugate }$$ Therefore, $y_1$ does not contain secular terms if and only if the coefficents are zero, which implies that $A(\tau)$ must satisfy the ODE $$2i\frac{dA}{d\tau} + \delta A(\tau) + \frac12\overline{A(\tau)} = 0.$$ Most other values of $k$ produce the simpler $$2i\frac{dA}{d\tau} + \delta A(\tau)=0,$$ which has the solution proprtional to $e^{i\delta\tau/2}$.

Often, solvability conditions produce new differential equations, but this is not inherently a problem. These solvability conditions by construction neglect what happens on the small scales, which in a computational setting is a huge boon, since the calculation of small scale effects can often take the most program time (since each step must be done many times) with marginal gains to accuracy. 
